In [1]:
import glob
from kid_readout.measurement.io import nc
from kid_readout.measurement import mmw_source_sweep, temperature

In [2]:
nc_filename = '/data/readout/2016-04-20_044055_off_on_modulated_10_dB_dac.nc'
#nc_filename = '/data/readout/2016-04-23_231033_off_on_modulated_broadband_2_dB_dac.nc'
ncf = nc.NCFile(nc_filename)

In [3]:
ncf.measurement_names()

[u'SweepArray0', u'StreamArray1', u'StreamArray2']

In [4]:
mmw = mmw_source_sweep.MMWSweepOnMod(sweep=ncf.SweepArray0,
                                     off_stream=None,  #ncf.StreamArray1,
                                     on_stream=ncf.StreamArray1,  # ncf.StreamArray2,
                                     mod_stream=ncf.StreamArray2)  #ncf.StreamArray3)

AttributeError: 'MMWSweepOnMod' object has no attribute 'add_measurement'

In [ ]:
df = mmw.to_dataframe()

In [ ]:
# from Glenn's notebook
def process_one(fn):
    ncf = nc.NCFile(fn)
    off_stream = None
    on_stream = None
    sweep = None
    mod_stream = None
    for name in ncf.measurement_names():
        meas = ncf.read(name)
        if name.startswith('Sweep'):
            sweep = meas
        else:
            modulation = meas.stream(0).roach_state.modulation_output 
            #print name, modulation
            if modulation == 0:
                on_stream = meas
            elif modulation == 1:
                off_stream = meas
            elif modulation == 2:
                mod_stream = meas
    if on_stream is None:
        on_stream = off_stream
        off_stream = None
    if on_stream is not None:
        meas = mmw_source_sweep.MMWSweepOnMod(ncf.SweepArray0,on_stream = on_stream, off_stream = off_stream, mod_stream = mod_stream)
        temperature.add_temperature(meas,'starcryo',recursive=True)
        rows = meas.to_dataframe(add_origin=True)
    else:
        rows = None
    ncf.close()
    return rows

In [ ]:
df = process_one(nc_filename)